In [76]:
import numpy as np
import plotly.express as px

from ipa.GenerateReferencePoints import generate_points
from ipa.intersection import find_bad_indicesREF, find_bad_limits
from ipa.scalarization import ReferencePointASF

In [62]:
# Data on a 3d simplex, 1000 points

# Reference points
ref_points = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

# Data
n_points = 10000
data = np.random.rand(n_points, 3)
data /= data.sum(axis=1)[:, None]

# make a hole in the front by rmoving points within a sphere

center = np.array([1, 1, 1]) / 3
radius = 0.5
mask = np.linalg.norm(data - center, axis=1) > radius

data = data[mask]

fig = px.scatter_3d(x=data[:, 0], y=data[:, 1], z=data[:, 2], title="Data on a 3d simplex")
fig

In [63]:
# ASF

refp = np.array([1,1,1])
asf = ReferencePointASF(preferential_factors=1, nadir=np.array([1, 1, 1]), utopian_point=np.array([0, 0, 0]))

values = asf(data, refp)
soln = data[np.argmin(values, axis=0)]

In [96]:
fig = px.scatter_3d(x=data[:, 0], y=data[:, 1], z=data[:, 2], title="Data on a 3d simplex")
fig.add_scatter3d(x=[soln[0]], y=[soln[1]], z=[soln[2]], mode="markers", marker=dict(size=10, color="red"))
# Add ideal and nadir points

fig.add_scatter3d(x=[0, 1], y=[0, 1], z=[0, 1], mode="lines+markers", marker=dict(size=5, color="black"))

In [97]:
_, all_ref_points = generate_points(10000, 3)

bad_points, _, _ = find_bad_indicesREF(
    solution = soln,
    ref_point=refp,
    reference_points=all_ref_points,
    thickness=0,
)

bmin, bmax = find_bad_limits(
    solution = soln,
    ref_point=refp,
    threshold=0,
)


fig.add_scatter3d(
    x=all_ref_points[:, 0],
    y=all_ref_points[:, 1],
    z=all_ref_points[:, 2],
    mode="markers",
    marker=dict(size=2, color=bad_points.astype(int),opacity=0.1),
)

# Add box made by bmin and bmax
x = [bmin[0], bmax[0], bmax[0], bmin[0], bmin[0], bmin[0], bmax[0], bmax[0], bmin[0], bmin[0]]
y = [bmin[1], bmin[1], bmax[1], bmax[1], bmin[1], bmin[1], bmax[1], bmax[1], bmin[1], bmin[1]]
z = [bmin[2], bmin[2], bmin[2], bmin[2], bmin[2], bmax[2], bmax[2], bmax[2], bmax[2], bmax[2]]

fig.add_scatter3d(x=x, y=y, z=z, mode="lines", line=dict(color="black"))
fig.add_scatter3d(x=[bmin[0]], y=[bmin[1]], z=[bmin[2]], mode="markers", marker=dict(size=5, color="black"), name="Projection of optimal <br>solution on the compromise line")

# Change projection
fig.update_layout(scene=dict(camera=dict(projection=dict(type="orthographic"))))

# remove legend
fig.update_layout(showlegend=False)

In [80]:
bmin, bmax


(array([0.7060664, 0.7060664, 0.7060664]),
 array([0.00169531, 0.29223829, 0.7060664 ]))